In [2]:
import tensorflow as tf
import keras
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os,sys
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import BatchNormalization, Activation, Input, Dropout, ZeroPadding2D, Lambda

# might be keras.layers.merge import Add Concatenate
from keras.layers import Concatenate, Add
from keras.models import Model
from keras.optimizers import Adam

In [14]:
def BatchNorm():
    return BatchNormalization(momentum=0.95, epsilon=1e-5)


In [15]:

def common_skip(prev, num_filters, atrous_rate, 
                kernel_size, stride_tuple, name):
    """
    The common ResNet block shared by the identity block
    and the convolutional block. Both of those blocks share
    this common functionality.
    """

    prev = BatchNorm()(prev)
    prev = Activation('relu')(prev)
    # prev = ZeroPadding2D(padding=(pad, pad))(prev)
    
    """Syntax:
    keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), 
                    padding='valid', data_format=None, dilation_rate=(1, 1),
                    activation=None, use_bias=True)
    """
    if name=="halve_feature_map":
        prev = Conv2D(num_filters, kernel_size=kernel_size, 
                      strides=(2*stride_tuple[0],2*stride_tuple[1]),
                      dilation_rate=atrous_rate,
                      use_bias=False)(prev)
    else:
        prev = Conv2D(num_filters, kernel_size=kernel_size, 
                      strides=stride_tuple, dilation_rate=atrous_rate,
                      use_bias=False)(prev)

    prev = BatchNorm()(prev)
    prev = Activation('relu')(prev)
    # dropout rate is 10%
    prev = Dropout(rate=0.1)
    prev = Conv2D(num_filters, kernel_size=kernel_size, 
                  strides=stride_tuple, dilation_rate=atrous_rate,
                  use_bias=False)(prev)
    
    # I think this should be followed by BatchNorm and an activation
    return prev

def convolution_branch(num_filters, kernel_size, prev):

    prev = Conv2D(num_filters, kernel_size=(1, 1), strides=(2, 2),
                      use_bias=False)(prev)

    prev = BatchNorm()(prev)
    return prev


def empty_branch(prev):
    return prev

In [ ]:
def convolutional_resnet_block(prev_layer, num_filters, kernel_size,
                               stride_tuple, name, atrous_rate=1):
   
    prev_layer = Activation('relu')(prev_layer)
    block_1 = common_skip(prev_layer, num_filters, kernel_size=kernel_size, 
                          stride_tuple=stride_tuple,
                          atrous_rate=atrous_rate, name)

    block_2 = convolution_branch(num_filters=num_filters,
                                 kernel_size=kernel_size, 
                                 prev_layer=prev_layer)
    added = Add()([block_1, block_2])
    
    return added
    

In [11]:
def identity_resnet_block(prev_layer, num_filters, kernel_size,
                          stride_tuple, name, atrous_rate=1):
    
    prev_layer = Activation('relu')(prev_layer)

    block_1 = common_skip(prev_layer, num_filters, kernel_size=kernel_size, 
                          stride_tuple=stride_tuple,
                          atrous_rate=atrous_rate, name)
    
    block_2 = empty_branch(prev_layer)
    
    added = Add()([block_1, block_2])
    
    return added


In [12]:
def PSPNet_ResNet(input_layer):
    
    x = Conv2D(16, (3, 3), strides=(1, 1), padding='same',
                  use_bias=False)(input_layer)
    
    x = identity_resnet_block(x, num_filters=16, kernel_size=(3,3),
                              stride_tuple=(1,1), name="identity",
                              atrous_rate=1)
    
    x = convolutional_resnet_block(x, num_filters=32, kernel_size=(3,3),
                                   stride_tuple=(1,1), name="halve_feature_map",
                                   atrous_rate=1)
    
    x = convolutional_resnet_block(x, num_filters=64, kernel_size=(3,3),
                                   stride_tuple=(1,1), name="halve_feature_map", 
                                   atrous_rate=1)
    
    x = identity_resnet_block(x, num_filters=64, kernel_size=(3,3),
                              stride_tuple=(1,1), name="identity",
                              atrous_rate=1)
    
    x = convolutional_resnet_block(x, num_filters=128, kernel_size=(3,3),
                                   stride_tuple=(1,1), name="halve_feature_map", 
                                   atrous_rate=1)
    
    x = identity_resnet_block(x, num_filters=128, kernel_size=(3,3),
                              stride_tuple=(1,1), name="identity",
                              atrous_rate=1)
    
    """ dilated/atrous convolutional ResNet block starts here"""
    
    x = convolutional_resnet_block(x, num_filters=256, kernel_size=(3,3),
                                   stride_tuple=(1,1), name="full_feature_map", 
                                   atrous_rate=2)
    
    x = identity_resnet_block(x, num_filters=256, kernel_size=(3,3),
                              stride_tuple=(1,1), name="identity",
                              atrous_rate=2)
    
    x = convolutional_resnet_block(x, num_filters=512, kernel_size=(3,3),
                                   stride_tuple=(1,1), name="full_feature_map", 
                                   atrous_rate=4)
    
    x = identity_resnet_block(x, num_filters=512, kernel_size=(3,3),
                              stride_tuple=(1,1), name="identity",
                              atrous_rate=4)
    #Syntax:
    #keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), 
    #                padding='valid', data_format=None, dilation_rate=(1, 1),
    #                activation=None, use_bias=True)
    
    x = Conv2D(filters=512,kernel_size=(3,3),
               strides=(1,1), padding='valid',dilation_rate=2)(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=512,kernel_size=(3,3),
               strides=(1,1), padding='valid',dilation_rate=2)(x)
    x = Activation('relu')(x)
    
    """End of dilated convolution block"""
    
    x = Conv2D(filters=512,kernel_size=(3,3),
               strides=(1,1), padding='valid',dilation_rate=1)(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=512,kernel_size=(3,3),
               strides=(1,1), padding='valid',dilation_rate=1)(x)
    x = Activation('relu')(x)
    
    
    """Spatial Pyramid Pooling"""
    